# Settings

In [ ]:
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

In [ ]:
!pip install ratsnlp

In [ ]:
import torch
from ratsnlp.nlpbook.qa import QATrainArguments
args = QATrainArguments(
    pretrained_model_name='beomi/kcbert-base',
    downstream_corpus_name='korquad-v1',
    downstream_corpus_root_dir='.data/Korpora',
    downstream_model_dir='.checkpoint-qa',
    max_seq_length=128, # 입력 문장 최대 길이(질문과 지문 모두 포함)
    max_query_length=32, # 질문 최대 길이
    doc_stride=64, # 지문에서 몇 개 토큰을 슬라이딩해가면서 데이터를 늘릴지 결정
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=52-5,
    epochs=3,
    tpu_cores = 0 if torch.cuda.is_available() else 8,
    seed=7,
)

In [ ]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

# 말뭉치 다운로드

In [ ]:
nlpbook.download_downstream_dataset(args)

# 토크나이저 설정

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

# 데이터 전처리

In [ ]:
from ratsnlp.nlpbook.qa import KorQuADV1Corpus, QADataset
corpus = KorQuADV1Corpus()
train_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode='train',
)

In [ ]:
train_dataset[0]

In [ ]:
train_dataset[1]

# 데이터 로더 구축

In [ ]:
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

In [ ]:
from torch.utils.data import SequentialSampler
val_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode='val',
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

# 모델 로드

In [ ]:
from transformers import BertConfig, BertForQuestionAnswering
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
)
model = BertForQuestionAnswering.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)

# 모델 학습

In [ ]:
from ratsnlp.nlpbook.qa import QATask
task = QATask(model, args)

In [ ]:
trainer = nlpbook.get_trainer(args)

In [ ]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

# 모델 출력 및 후처리

In [ ]:
def inference_fn(question, context):
    if question and context:
        # question 토큰화 및 인덱싱
        truncated_query = tokenizer.encode(
            question,
            add_special_tokens=False,
            truncation=True,
            max_length=dargs.max_query_length, # max_length 초과 시 자름
        )
        # truncated_query를 context와 함께 토큰화 및 인덱싱
        inputs = tokenizer.encode_plus(
            text=truncated_query,
            text_pair=context,
            truncation='only_second', # 전체 길이가 max_length 초과 시 자름
            padding='max_length',
            max_length=dargs.max_seq_length,
            return_token_type_ids=True,
        )
        with torch.no_grad():
            outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})

            # 정답의 시작 위치와 관련된 로짓(outputs.start_logits)에서 가장 큰 값이 가리키는 토큰 위치
            start_pred = outputs.start_logits.argmax(dim=-1).item()
            # 정답의 끝 위치와 관련된 로짓(outputs.end_logits)에서 가장 큰 값이 가리키는 토큰 위치
            end_pred = outputs.end_logits.argmax(dim=-1).item()
            # 정답 시작부터 끝까지의 토큰을 연결하여 정답 생성
            pred_text = tokenizer.decode(inputs['input_ids'][start_pred:end_pred+1])
    else:
        pred_text = ''
    return {
        'question': question,
        'context': context,
        'answer': pred_text
    }

# 검증

In [ ]:
#from ratsnlp.nlpbook.qa import get_web_service_app
#app = get_web_service_app(inference_fn)
#app.run()

In [ ]:
context = "한강대교(漢江大橋)는 서울특별시 용산구 이촌동에 있는 용산구 한강로3가와 동작구 본동 사이를 잇는 총연장 1,005m의 길이의 교량(다리)이다. 한강에 놓인 최초의 도로 교량으로, 제1한강교라고 불렸다. 1917년 개통된 뒤 몇 차례의 수난을 거쳐 지금에 이른다. 다리 아래로는 노들섬이 있다. 과거에는 국도 제1호선이 이 다리를 통하여 서울로 연결되었었다."
question = "한강대교 아래에는 어떤 섬이 있는가?"
inference_fn(question=question, context=context)